![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# ONCOLOGY MODELS

This notebook includes details about different kinds of pretrained models to extract oncology-related information from clinical texts, together with examples of each type of model.

## Setup

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## List of Pretrained Models

In [0]:
df = pd.DataFrame()
for model_type in ['MedicalNerModel', 'BertForTokenClassification', 'RelationExtractionModel', 'RelationExtractionDLModel', 'AssertionDLModel']:
    model_list = list(set([model[0] for model in medical.InternalResourceDownloader.returnPrivateModels(model_type) if 'oncology' in model[0]]))
    if len(model_list) > 0:
      df = pd.concat([df, pd.DataFrame(model_list, columns = [model_type])], axis = 1)
    
df.fillna('')

Out[34]:

MedicalNerModel 
 RelationExtractionModel 
 RelationExtractionDLModel 
 AssertionDLModel 
 
 
 
 
 0 
 ner_oncology_biomarker 
 re_oncology_wip 
 redl_oncology_location_biobert_wip 
 assertion_oncology_family_history_wip 
 
 
 1 
 ner_oncology_biomarker_wip 
 re_oncology_test_result_wip 
 redl_oncology_temporal_biobert_wip 
 assertion_oncology_response_to_treatment_wip 
 
 
 2 
 ner_oncology_therapy_wip 
 re_oncology_biomarker_result_wip 
 redl_oncology_biomarker_result_biobert_wip 
 assertion_oncology_problem_wip 
 
 
 3 
 ner_oncology_unspecific_posology 
 re_oncology_location_wip 
 redl_oncology_granular_biobert_wip 
 assertion_oncology_smoking_status_wip 
 
 
 4 
 ner_oncology_anatomy_general 
 re_oncology_temporal_wip 
 redl_oncology_size_biobert_wip 
 assertion_oncology_demographic_binary_wip 
 
 
 5 
 ner_oncology_response_to_treatment 
 re_oncology_granular_wip 
 redl_oncology_test_result_biobert_wip 
 assertion_oncology_test_binary_wip 
 
 
 6 
 ner_oncology_tnm 
 re_oncology_size_wip 
 redl_oncology_biobert_wip 
 assertion_oncology_treatment_binary_wip 
 
 
 7 
 ner_oncology_unspecific_posology_wip 
 
 
 assertion_oncology_wip 
 
 
 8 
 ner_oncology_wip 
 
 
 
 
 
 9 
 ner_oncology_test_wip 
 
 
 
 
 
 10 
 ner_oncology_demographics_wip 
 
 
 
 
 
 11 
 ner_oncology_tnm_wip 
 
 
 
 
 
 12 
 ner_oncology_diagnosis_wip 
 
 
 
 
 
 13 
 ner_oncology_response_to_treatment_wip 
 
 
 
 
 
 14 
 ner_oncology_anatomy_general_wip 
 
 
 
 
 
 15 
 ner_oncology_therapy 
 
 
 
 
 
 16 
 ner_oncology_posology_wip 
 
 
 
 
 
 17 
 ner_oncology_test 
 
 
 
 
 
 18 
 ner_oncology_posology 
 
 
 
 
 
 19 
 ner_oncology_diagnosis 
 
 
 
 
 
 20 
 ner_oncology_demographics 
 
 
 
 
 
 21 
 ner_oncology 
 
 
 
 
 
 22 
 ner_oncology_anatomy_granular 
 
 
 
 
 
 23 
 ner_oncology_anatomy_granular_wip

## NER Models

The NER models from the list include different entity groups and levels of granularity. If you want to extract as much information as possible from oncology texts, then ner_oncology_wip is the best option for you, as it is the most general and granular model. But you may want to use other models depending on your needs (for instance, if you need to extract information related with staging, ner_oncology_tnm_wip would be the most suitable model).

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setSplitChars(["-", "\/"])

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# ner_oncology_wip

ner_oncology_wip = medical.NerModel.pretrained("ner_oncology_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_wip")\

ner_oncology_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_wip"])\
    .setOutputCol("ner_oncology_wip_chunk")

# ner_oncology_tnm_wip

ner_oncology_tnm_wip = medical.NerModel.pretrained("ner_oncology_tnm_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_tnm_wip")\

ner_oncology_tnm_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_tnm_wip"])\
    .setOutputCol("ner_oncology_tnm_wip_chunk")

# # ner_oncology_biomarker_wip

ner_oncology_biomarker_wip = medical.NerModel.pretrained("ner_oncology_biomarker_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_biomarker_wip")\

ner_oncology_biomarker_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_biomarker_wip"])\
    .setOutputCol("ner_oncology_biomarker_wip_chunk")

ner_stages = [document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_oncology_wip,
    ner_oncology_wip_converter,
    ner_oncology_tnm_wip,
    ner_oncology_tnm_wip_converter,
    ner_oncology_biomarker_wip,
    ner_oncology_biomarker_wip_converter]

ner_pipeline = nlp.Pipeline(stages=ner_stages)

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_oncology_wip download started this may take some time.
[ | ][OK!]
ner_oncology_tnm_wip download started this may take some time.
[ | ][OK!]
ner_oncology_biomarker_wip download started this may take some time.
[ | ][OK!]

In [0]:
ner_oncology_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_wip.getClasses() if label != 'O']))

len(ner_oncology_wip_labels)

Out[36]: 49

In [0]:
label_df = pd.DataFrame()
for column in range((len(ner_oncology_wip_labels)//10)+1):
  label_df = pd.concat([label_df, pd.DataFrame(ner_oncology_wip_labels, columns = [''])[column*10:(column+1)*10].reset_index(drop= True)], axis = 1)

label_df.fillna('')

Out[37]:

0 
 Biomarker 
 Site_Liver 
 Hormonal_Therapy 
 Chemotherapy 
 Death_Entity 
 
 
 1 
 Grade 
 Date 
 Cycle_Day 
 Site_Lung 
 Site_Brain 
 
 
 2 
 Pathology_Result 
 Imaging_Test 
 Age 
 Site_Breast 
 Invasion 
 
 
 3 
 Tumor_Finding 
 Cycle_Number 
 Relative_Date 
 Gender 
 Histological_Type 
 
 
 4 
 Unspecific_Therapy 
 Immunotherapy 
 Cycle_Count 
 Site_Other_Body_Part 
 Metastasis 
 
 
 5 
 Duration 
 Radiation_Dose 
 Cancer_Dx 
 Response_To_Treatment 
 Tumor_Size 
 
 
 6 
 Pathology_Test 
 Site_Lymph_Node 
 Direction 
 Frequency 
 Radiotherapy 
 
 
 7 
 Smoking_Status 
 Targeted_Therapy 
 Cancer_Score 
 Cancer_Surgery 
 Biomarker_Result 
 
 
 8 
 Oncogene 
 Performance_Status 
 Dosage 
 Race_Ethnicity 
 Staging 
 
 
 9 
 Line_Of_Therapy 
 Adenopathy 
 Route 
 Site_Bone

In [0]:
ner_oncology_tnm_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_tnm_wip.getClasses() if label != 'O']))

print(ner_oncology_tnm_wip_labels)

['Tumor_Description', 'Cancer_Dx', 'Tumor', 'Metastasis', 'Lymph_Node', 'Staging', 'Lymph_Node_Modifier']

In [0]:
ner_oncology_biomarker_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_biomarker_wip.getClasses() if label != 'O']))

print(ner_oncology_biomarker_wip_labels)

['Biomarker_Result', 'Biomarker']

In [0]:
sample_text_1 = '''A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total anterior hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma. Neoadjuvant chemotherapy with the regimens of Taxotere (75 mg/m2), Epirubicin (75 mg/m2), and Cyclophosphamide (500 mg/m2) was given for 6 cycles with poor response, followed by a modified radical mastectomy (MRM) with dissection of axillary lymph nodes and skin grafting. Postoperatively, radiotherapy was done with 5000 cGy in 25 fractions. The histopathologic examination revealed a metaplastic carcinoma with squamous differentiation associated with adenomyoepithelioma. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin (AE1/AE3) stain, and myoepithelial markers, including cytokeratin 5/6 (CK 5/6), p63, and S100 stains. Expressions of hormone receptors, including ER, PR, and Her-2/Neu, were all negative. The dissected axillary lymph nodes showed metastastic carcinoma with negative hormone receptors in 3 nodes. The patient was staged as pT3N1aM0, with histologic tumor grade III.'''

sample_text_2 = '''She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.'''

sample_text_3 = '''In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9, CD10, CD13, CD19, CD20, CD34, CD38, CD58, CD66c, CD123, HLA-DR, cCD79a, and TdT on flow cytometry.

Measurements of serum tumor markers showed elevated level of cytokeratin 19 fragment (Cyfra21-1: 4.77 ng/mL), neuron-specific enolase (NSE: 19.60 ng/mL), and squamous cell carcinoma antigen (SCCA: 2.58 ng/mL). The results were negative for serum carbohydrate antigen 125 (CA125), carcinoembryonic antigen (CEA) and vascular endothelial growth factor (VEGF). Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1 (+ 80% tumor cells), and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Molecular analysis indicated no EGFR mutation or ROS1 fusion.'''

In [0]:
data = spark.createDataFrame(pd.DataFrame([sample_text_1, sample_text_2, sample_text_3], columns = ['text']))

In [0]:
results = ner_model.transform(data).collect()

In [0]:
visualiser = nlp.viz.NerVisualizer()

In [0]:
ner_html_1 = visualiser.display(results[0], label_col='ner_oncology_wip_chunk', return_html=True)

displayHTML(ner_html_1)

A 65-year-old Age woman Gender had a history of debulking surgery Cancer_Surgery , bilateral Direction oophorectomy Cancer_Surgery with omentectomy Cancer_Surgery , total anterior hysterectomy Cancer_Surgery with radical pelvic lymph nodes dissection Cancer_Surgery due to ovarian carcinoma Cancer_Dx ( mucinous-type Histological_Type carcinoma Cancer_Dx , stage Ic Staging ) 1 year ago Relative_Date . Patient's medical compliance was poor and failed to complete her Gender chemotherapy Chemotherapy ( cyclophosphamide Chemotherapy 750 mg/m2 Dosage , carboplatin Chemotherapy 300 mg/m2 Dosage ). Recently, she Gender noted a palpable right Direction breast Site_Breast mass Tumor_Finding , 15 cm Tumor_Size in size which nearly occupied the whole right Direction breast Site_Breast in 2 months Relative_Date . Core needle biopsy Pathology_Test revealed metaplastic Histological_Type carcinoma Cancer_Dx . Neoadjuvant chemotherapy Chemotherapy with the regimens of Taxotere Chemotherapy ( 75 mg/m2 Dosage ), Epirubicin Chemotherapy ( 75 mg/m2 Dosage ), and Cyclophosphamide Chemotherapy ( 500 mg/m2 Dosage ) was given for 6 cycles Cycle_Count with poor response Response_To_Treatment , followed by a modified radical mastectomy Cancer_Surgery ( MRM Imaging_Test ) with dissection Cancer_Surgery of axillary lymph nodes Site_Lymph_Node and skin grafting Cancer_Surgery . Postoperatively, radiotherapy Radiotherapy was done with 5000 cGy Radiation_Dose in 25 fractions Cycle_Count . The histopathologic examination Pathology_Test revealed a metaplastic Histological_Type carcinoma Cancer_Dx with squamous differentiation Pathology_Result associated with adenomyoepithelioma Cancer_Dx . Immunohistochemistry study Pathology_Test showed that the tumor cells Pathology_Result are positive Biomarker_Result for epithelial markers-cytokeratin Biomarker ( AE1/AE3 Biomarker ) stain, and myoepithelial markers Biomarker , including cytokeratin 5/6 Biomarker ( CK 5/6 Biomarker ), p63 Biomarker , and S100 Biomarker stains. Expressions of hormone receptors Biomarker , including ER Biomarker , PR Biomarker , and Her Gender - 2/Neu Biomarker , were all negative Biomarker_Result . The dissected axillary lymph nodes Site_Lymph_Node showed metastastic Metastasis carcinoma Cancer_Dx with negative Biomarker_Result hormone receptors Biomarker in 3 nodes Site_Lymph_Node . The patient was staged as pT3N1aM0 Staging , with histologic tumor Histological_Type grade III Grade .

In [0]:
ner_html_2 = visualiser.display(results[1], label_col='ner_oncology_tnm_wip_chunk', return_html=True)

displayHTML(ner_html_2)

She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass Tumor . A Pap smear performed one month later was positive for atypical glandular cells Tumor_Description suspicious for adenocarcinoma Cancer_Dx . The pathologic specimen showed extension Tumor_Description of the tumor Tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged Lymph_Node_Modifier lymph nodes Lymph_Node . The final pathologic diagnosis of the tumor Tumor was stage IIIC Staging papillary Tumor_Description serous Tumor_Description ovarian adenocarcinoma Cancer_Dx . Two months later, the patient was diagnosed with lung metastases Metastasis .

In [0]:
ner_html_3 = visualiser.display(results[2], label_col='ner_oncology_biomarker_wip_chunk', return_html=True)

displayHTML(ner_html_3)

In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive Biomarker_Result for CD9 Biomarker , CD10 Biomarker , CD13 Biomarker , CD19 Biomarker , CD20 Biomarker , CD34 Biomarker , CD38 Biomarker , CD58 Biomarker , CD66c Biomarker , CD123 Biomarker , HLA-DR Biomarker , cCD79a Biomarker , and TdT Biomarker on flow cytometry. Measurements of serum tumor markers Biomarker showed elevated Biomarker_Result level of cytokeratin 19 fragment Biomarker ( Cyfra21-1 Biomarker : 4.77 ng/mL Biomarker_Result ), neuron-specific enolase Biomarker ( NSE Biomarker : 19.60 ng/mL Biomarker_Result ), and squamous cell carcinoma antigen Biomarker ( SCCA Biomarker : 2.58 ng/mL Biomarker_Result ). The results were negative Biomarker_Result for serum carbohydrate antigen 125 Biomarker ( CA125 Biomarker ), carcinoembryonic antigen Biomarker ( CEA Biomarker ) and vascular endothelial growth factor Biomarker ( VEGF Biomarker ). Immunohistochemical staining showed positive Biomarker_Result staining for CK5/6 Biomarker , P40 Biomarker and PD-L1 Biomarker ( + Biomarker_Result 80% tumor cells), and negative Biomarker_Result staining for TTF-1 Biomarker , PD-1 Biomarker and weakly positive Biomarker_Result staining for ALK Biomarker . Molecular analysis indicated no EGFR mutation Biomarker or ROS1 Biomarker fusion.

## Relation Extraction Models

RE Models are used to link entities that are related. For oncology entities, you can use general models (such as re_oncology_granular_wip) or you can select a specific model depending on your needs (e.g. re_oncology_size_wip to link tumors and their sizes, or re_oncology_biomarker_result_wip to link biomarkers and their results).

In [0]:
pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel.pretrained("dependency_conllu", "en") \
    .setInputCols(["sentence", "pos_tags", "token"]) \
    .setOutputCol("dependencies")

re_oncology_granular_wip = medical.RelationExtractionModel.pretrained("re_oncology_granular_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_granular_wip") \
    .setRelationPairs(['Date-Cancer_Dx', 'Cancer_Dx-Date', 'Tumor_Finding-Site_Breast', 'Site_Breast-Tumor_Finding',
                       'Relative_Date-Tumor_Finding', 'Tumor_Fiding-Relative_Date', 'Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding',
                       'Pathology_Test-Cancer_Dx', 'Cancer_Dx-Pathology_Test']) \
    .setMaxSyntacticDistance(10)    

re_oncology_size_wip = medical.RelationExtractionModel.pretrained("re_oncology_size_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_size_wip") \
    .setRelationPairs(['Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding']) \
    .setMaxSyntacticDistance(10)    

re_oncology_biomarker_result_wip = medical.RelationExtractionModel.pretrained("re_oncology_biomarker_result_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_biomarker_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_biomarker_result_wip") \
    .setRelationPairs(['Biomarker-Biomarker_Result', 'Biomarker_Result-Biomarker']) \
    .setMaxSyntacticDistance(10)      

re_stages = ner_stages + [pos_tagger, dependency_parser, re_oncology_granular_wip, re_oncology_size_wip, re_oncology_biomarker_result_wip]

re_pipeline = nlp.Pipeline(stages=re_stages)

re_model = re_pipeline.fit(empty_data)

pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
re_oncology_granular_wip download started this may take some time.
Approximate size to download 261 KB
[ | ][OK!]
re_oncology_size_wip download started this may take some time.
Approximate size to download 261.3 KB
[ | ][OK!]
re_oncology_biomarker_result_wip download started this may take some time.
Approximate size to download 259.6 KB
[ | ][OK!]

In [0]:
sample_text_4 = '''Two years ago, she noted a palpable right breast mass, 15 cm in size. Core needle biopsy revealed metaplastic carcinoma.'''

sample_text_5 = '''The patient presented a 2 cm mass in her left breast, and the tumor in her other breast was 3 cm long.'''

sample_text_6 = '''Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1, and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin and myoepithelial markers, including cytokeratin 5/6, p63, and S100 stains.'''

In [0]:
re_data = spark.createDataFrame(pd.DataFrame([sample_text_4, sample_text_5, sample_text_6], columns = ['text']))

In [0]:
re_results = re_model.transform(re_data).collect()

In [0]:
re_visualiser = nlp.viz.RelationExtractionVisualizer()

In [0]:
re_html_1 = re_visualiser.display(re_results[0], relation_col='re_oncology_granular_wip', return_html=True)

displayHTML(re_html_1)

Two years ago RELATIVE_DATE , she noted a palpable right breast SITE_BREAST mass TUMOR_FINDING , 15 cm TUMOR_SIZE in size. Core needle biopsy PATHOLOGY_TEST revealed metaplastic carcinoma CANCER_DX . <polyline fill="none" points="601.0,55.0 600.204081632653,53.79607986468223 599.4081632653061,52.593128713376224 598.6122448979592,51.39211553009376 597.8163265306123,50.194009298846574 597.0204081632654,48.999779003646445 596.2244897959184,47.810393628505125 595.4285714285716,46.626822157434425 594.6326530612245,45.450033574446024 593.8367346938776,44.280996863551756 593.0408163265305,43.12068100876335 592.2448979591836,41.97005499409259 591.4489795918366,40.83008780355124 590.6530612244898,39.70174842115105 589.8571428571428,38.586005830903794 589.061224489796,37.48382901682123 588.2653061224489,36.396186962915124 587.4693877551022,35.32404865319723 586.6734693877551,34.26838307167932 585.8775510204081,33.23015920237316 585.0816326530611,32.21034602929051 584.2857142857142,31.209912536443145 583.4897959183673,30.229827707842823 582.6938775510204,29.271060527501298 581.8979591836735,28.334579979430348 581.1020408163266,27.421355047641715 580.3061224489795,26.532354716147186 579.5102040816327,25.668547968958514 578.7142857142858,24.83090379008747 577.9183673469388,24.020391163545803 577.1224489795918,23.237979073345294 576.3265306122449,22.484636503497697 575.530612244898,21.761332438014776 574.734693877551,21.069035860908297 573.9387755102041,20.408715756190023 573.1428571428572,19.781341107871725 572.3469387755102,19.187880899965155 571.5510204081633,18.629304116482082 570.7551020408164,18.106579741434267 569.9591836734694,17.620676758833483 569.1632653061224,17.172564152691482 568.3673469387755,16.76321090702004 567.5714285714287,16.393586005830905 566.7755102040817,16.064658433135854 565.9795918367347,15.77739717294665 565.1836734693877,15.53277120927505 564.3877551020408,15.33174952613282 563.5918367346939,15.175301107531727 562.795918367347,15.064394937483533 562.0,15.0 562.0,15.0 561.204081632653,14.982762284422305 560.408163265306,15.012035801409274 559.6122448979593,15.086851566949145 558.8163265306122,15.20624059703015 558.0204081632652,15.369233907640522 557.2244897959183,15.574862514768505 556.4285714285716,15.822157434402333 555.6326530612246,16.11014968253024 554.8367346938776,16.43787027514046 554.0408163265305,16.80435022822123 553.2448979591836,17.208620557760792 552.4489795918367,17.64971227974738 551.6530612244898,18.126656410169232 550.8571428571429,18.638483965014576 550.061224489796,19.184225960271654 549.265306122449,19.762913411928704 548.469387755102,20.37357733597396 547.6734693877551,21.01524874839565 546.8775510204082,21.686958665182026 546.0816326530611,22.387738102321304 545.2857142857141,23.116618075801746 544.4897959183673,23.87262960161157 543.6938775510204,24.654803695739016 542.8979591836735,25.462171374172325 542.1020408163265,26.293763652899724 541.3061224489796,27.148611547909454 540.5102040816327,28.02574607518976 539.7142857142858,28.92419825072886 538.9183673469388,29.842999090515008 538.1224489795919,30.78117961053642 537.3265306122449,31.737770826781354 536.530612244898,32.71180375523804 535.7346938775511,33.7023094118947 534.9387755102041,34.70831881273959 534.1428571428572,35.728862973760926 533.3469387755101,36.762972910946964 532.5510204081633,37.809679640285935 531.7551020408164,38.868014177766064 530.9591836734694,39.93700753937559 530.1632653061224,41.015690741102766 529.3673469387755,42.103094798935814 528.5714285714286,43.19825072886297 527.7755102040817,44.300189546872474 526.9795918367347,45.40794226895255 526.1836734693877,46.52053991109146 525.3877551020408,47.63701348927743 524.5918367346939,48.75639401949867 523.795918367347,49.877712517743454 523.0,51.0" stroke="#6E4130" stroke-width="1" /> is_location_of <polyline fill="none" points="693.0,55.0 692.1632653061224,53.79607986468223 691.3265306122449,52.593128713376224 690.4897959183675,51.39211553009376 689.6530612244898,50

In [0]:
re_html_2 = re_visualiser.display(re_results[1], relation_col='re_oncology_size_wip', return_html=True)

displayHTML(re_html_2)

The patient presented a 2 cm TUMOR_SIZE mass TUMOR_FINDING in her left breast, and the tumor TUMOR_FINDING in her other breast was 3 cm TUMOR_SIZE long. <polyline fill="none" points="357.0,55.0 356.3061224489796,53.79607986468223 355.61224489795916,52.593128713376224 354.91836734693885,51.39211553009376 354.22448979591843,50.194009298846574 353.53061224489795,48.999779003646445 352.8367346938775,47.810393628505125 352.1428571428573,46.626822157434425 351.44897959183675,45.450033574446024 350.7551020408164,44.280996863551756 350.0612244897959,43.12068100876335 349.3673469387754,41.97005499409259 348.67346938775506,40.83008780355124 347.9795918367347,39.70174842115105 347.2857142857143,38.586005830903794 346.59183673469397,37.48382901682123 345.8979591836735,36.396186962915124 345.2040816326531,35.32404865319723 344.51020408163265,34.26838307167932 343.81632653061223,33.23015920237316 343.1224489795918,32.21034602929051 342.42857142857133,31.209912536443145 341.734693877551,30.229827707842823 341.04081632653066,29.271060527501298 340.34693877551024,28.334579979430348 339.6530612244899,27.421355047641715 338.9591836734694,26.532354716147186 338.26530612244903,25.668547968958514 337.5714285714286,24.83090379008747 336.8775510204082,24.020391163545803 336.18367346938777,23.237979073345294 335.4897959183674,22.484636503497697 334.795918367347,21.761332438014776 334.1020408163265,21.069035860908297 333.40816326530614,20.408715756190023 332.7142857142857,19.781341107871725 332.0204081632653,19.187880899965155 331.32653061224494,18.629304116482082 330.63265306122446,18.106579741434267 329.9387755102041,17.620676758833483 329.24489795918373,17.172564152691482 328.55102040816325,16.76321090702004 327.8571428571429,16.393586005830905 327.16326530612247,16.064658433135854 326.46938775510205,15.77739717294665 325.7755102040817,15.53277120927505 325.0816326530612,15.33174952613282 324.38775510204084,15.175301107531727 323.6938775510204,15.064394937483533 323.0,15.0 323.0,15.0 322.3061224489796,14.982762284422305 321.61224489795916,15.012035801409274 320.91836734693885,15.086851566949145 320.2244897959183,15.20624059703015 319.53061224489795,15.369233907640522 318.83673469387753,15.574862514768505 318.1428571428572,15.822157434402333 317.4489795918368,16.11014968253024 316.7551020408163,16.43787027514046 316.0612244897958,16.80435022822123 315.3673469387754,17.208620557760792 314.673469387755,17.64971227974738 313.97959183673464,18.126656410169232 313.2857142857143,18.638483965014576 312.5918367346939,19.184225960271654 311.8979591836735,19.762913411928704 311.2040816326531,20.37357733597396 310.51020408163265,21.01524874839565 309.81632653061223,21.686958665182026 309.12244897959175,22.387738102321304 308.4285714285714,23.116618075801746 307.73469387755097,23.87262960161157 307.04081632653055,24.654803695739016 306.3469387755102,25.462171374172325 305.6530612244898,26.293763652899724 304.95918367346934,27.148611547909454 304.265306122449,28.02574607518976 303.57142857142856,28.92419825072886 302.87755102040813,29.842999090515008 302.1836734693877,30.78117961053642 301.48979591836735,31.737770826781354 300.7959183673469,32.71180375523804 300.1020408163265,33.7023094118947 299.40816326530614,34.70831881273959 298.7142857142857,35.728862973760926 298.0204081632653,36.762972910946964 297.3265306122449,37.809679640285935 296.63265306122446,38.868014177766064 295.9387755102041,39.93700753937559 295.24489795918373,41.015690741102766 294.55102040816325,42.103094798935814 293.8571428571429,43.19825072886297 293.1632653061224,44.300189546872474 292.46938775510205,45.40794226895255 291.7755102040816,46.52053991109146 291.08163265306126,47.63701348927743 290.3877551020408,48.75639401949867 289.69387755102036,49.877712517743454 289.0,51.0" stroke="#0EA924" stroke-width="1" /> is_size_of <polyline fill="none" points="80.0,155.0 86.12244897959184,154.33673469387753 92.24489795918365,153.6734693877551 98.36734693877553,153.01020408163268 104.48979591836735,

In [0]:
re_html_3 = re_visualiser.display(re_results[2], relation_col='re_oncology_biomarker_result_wip', return_html=True)

displayHTML(re_html_3)

Immunohistochemical staining showed positive BIOMARKER_RESULT staining for CK5/6 BIOMARKER , P40 BIOMARKER and PD-L1 BIOMARKER , and negative BIOMARKER_RESULT staining for TTF-1 BIOMARKER , PD-1 BIOMARKER and weakly positive BIOMARKER_RESULT staining for ALK BIOMARKER . Immunohistochemistry study showed that the tumor cells are positive BIOMARKER_RESULT for epithelial markers-cytokeratin BIOMARKER and myoepithelial markers BIOMARKER , including cytokeratin 5/6 BIOMARKER , p63 BIOMARKER , and S100 BIOMARKER stains. <polyline fill="none" points="634.0,55.0 631.8163265306122,53.79607986468223 629.6326530612245,52.593128713376224 627.4489795918367,51.39211553009376 625.265306122449,50.194009298846574 623.0816326530612,48.999779003646445 620.8979591836734,47.810393628505125 618.7142857142859,46.626822157434425 616.5306122448981,45.450033574446024 614.3469387755104,44.280996863551756 612.1632653061223,43.12068100876335 609.9795918367347,41.97005499409259 607.7959183673469,40.83008780355124 605.6122448979592,39.70174842115105 603.4285714285714,38.586005830903794 601.2448979591838,37.48382901682123 599.061224489796,36.396186962915124 596.8775510204083,35.32404865319723 594.6938775510205,34.26838307167932 592.5102040816327,33.23015920237316 590.3265306122449,32.21034602929051 588.1428571428571,31.209912536443145 585.9591836734695,30.229827707842823 583.7755102040816,29.271060527501298 581.591836734694,28.334579979430348 579.4081632653061,27.421355047641715 577.2244897959184,26.532354716147186 575.0408163265306,25.668547968958514 572.857142857143,24.83090379008747 570.6734693877552,24.020391163545803 568.4897959183675,23.237979073345294 566.3061224489797,22.484636503497697 564.1224489795918,21.761332438014776 561.9387755102041,21.069035860908297 559.7551020408164,20.408715756190023 557.5714285714287,19.781341107871725 555.387755102041,19.187880899965155 553.2040816326531,18.629304116482082 551.0204081632653,18.106579741434267 548.8367346938776,17.620676758833483 546.6530612244899,17.172564152691482 544.469387755102,16.76321090702004 542.2857142857143,16.393586005830905 540.1020408163265,16.064658433135854 537.9183673469388,15.77739717294665 535.7346938775511,15.53277120927505 533.5510204081633,15.33174952613282 531.3673469387755,15.175301107531727 529.1836734693877,15.064394937483533 527.0,15.0 527.0,15.0 524.8163265306122,14.982762284422305 522.6326530612243,15.012035801409274 520.4489795918367,15.086851566949145 518.2653061224489,15.20624059703015 516.0816326530612,15.369233907640522 513.8979591836735,15.574862514768505 511.71428571428584,15.822157434402333 509.530612244898,16.11014968253024 507.34693877551024,16.43787027514046 505.16326530612236,16.80435022822123 502.97959183673464,17.208620557760792 500.79591836734687,17.64971227974738 498.61224489795916,18.126656410169232 496.4285714285714,18.638483965014576 494.2448979591837,19.184225960271654 492.06122448979596,19.762913411928704 489.8775510204082,20.37357733597396 487.69387755102036,21.01524874839565 485.5102040816326,21.686958665182026 483.32653061224477,22.387738102321304 481.142857142857,23.116618075801746 478.95918367346945,23.87262960161157 476.77551020408157,24.654803695739016 474.59183673469386,25.462171374172325 472.4081632653061,26.293763652899724 470.2244897959184,27.148611547909454 468.0408163265306,28.02574607518976 465.85714285714283,28.92419825072886 463.6734693877551,29.842999090515008 461.48979591836735,30.78117961053642 459.30612244897964,31.737770826781354 457.12244897959187,32.71180375523804 454.9387755102041,33.7023094118947 452.75510204081627,34.70831881273959 450.57142857142856,35.728862973760926 448.3877551020408,36.762972910946964 446.204081632653,37.809679640285935 444.0204081632653,38.868014177766064 441.83673469387753,39.93700753937559 439.65306122448976,41.015690741102766 437.46938775510205,42.103094798935814 435.28571428571433,43.19825072886297 433.1020408163265,44.300189546872474 430.9183673469388,45.40794226895255 428.7346938775511,46.5205399110914

## Assertion Status Models

With assertion status models, you will be able to identify if entities included in texts are mentioned as something present, absent, hypothetical, possible, etc. You can either try using the general assertion_oncology_wip model, or other models that are recommended for specific entity groups (such as assertion_oncology_problem_wip, which should be used for problem entities like Cancer_Dx or Metastasis).

In [0]:
assertion_oncology_wip = medical.AssertionDLModel.pretrained("assertion_oncology_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_wip")

assertion_oncology_problem_wip = medical.AssertionDLModel.pretrained("assertion_oncology_problem_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_tnm_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_problem_wip")

assertion_oncology_treatment_binary_wip = medical.AssertionDLModel.pretrained("assertion_oncology_treatment_binary_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_treatment_binary_wip")

assertion_stages = ner_stages + [assertion_oncology_wip, assertion_oncology_problem_wip, assertion_oncology_treatment_binary_wip]

assertion_pipeline = nlp.Pipeline(stages=assertion_stages)

assertion_model = assertion_pipeline.fit(empty_data)

assertion_oncology_wip download started this may take some time.
[ | ][OK!]
assertion_oncology_problem_wip download started this may take some time.
[ | ][OK!]
assertion_oncology_treatment_binary_wip download started this may take some time.
[ | ][OK!]

In [0]:
sample_text_7 = 'The patient is suspected to have colorectal cancer. Family history is positive for other cancers. The result of the biopsy was positive. A CT scan was ordered to rule out metastases.'

sample_text_8 = 'The patient was diagnosed with breast cancer. She was suspected to have metastases in her lungs. Her family history is positive for ovarian cancer.'

sample_text_9 = 'The patient underwent a mastectomy. We recommend to start radiotherapy. The patient refused to chemotherapy.'

In [0]:
assertion_data = spark.createDataFrame(pd.DataFrame([sample_text_7, sample_text_8, sample_text_9], columns = ['text']))

In [0]:
assertion_results = assertion_model.transform(assertion_data).collect()

In [0]:
assertion_visualiser = nlp.viz.AssertionVisualizer()

In [0]:
assert_html_1 = assertion_visualiser.display(assertion_results[0], label_col ='ner_oncology_wip_chunk', assertion_col='assertion_oncology_wip', return_html=True)

displayHTML(assert_html_1)

The patient is suspected to have colorectal cancer Cancer_Dx Possible . Family history is positive for other cancers Cancer_Dx Family . The result of the biopsy Pathology_Test Past was positive. A CT scan Imaging_Test Past was ordered to rule out metastases Metastasis Possible .

In [0]:
assert_html_2 = assertion_visualiser.display(assertion_results[1], label_col ='ner_oncology_tnm_wip_chunk', assertion_col='assertion_oncology_problem_wip', return_html=True)

displayHTML(assert_html_2)

The patient was diagnosed with breast cancer Cancer_Dx Medical_History . She was suspected to have metastases Metastasis Possible in her lungs. Her family history is positive for ovarian cancer Cancer_Dx Family_History .

In [0]:
assert_html_3 = assertion_visualiser.display(assertion_results[2], label_col ='ner_oncology_wip_chunk', assertion_col='assertion_oncology_treatment_binary_wip', return_html=True)

displayHTML(assert_html_3)

The patient underwent a mastectomy Cancer_Surgery Present_Or_Past . We recommend to start radiotherapy Radiotherapy Hypothetical_Or_Absent . The patient refused to chemotherapy Chemotherapy Hypothetical_Or_Absent .